AES Cryptography

In [66]:
# Define libraries
import os
import re
import base64
import hashlib
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

In [67]:
class AESCipher:
    """
        An AESCipher object contains a key and a block_size. An AESCipher object also contains two methods for encryption and decryption that can be
        called from the object.
        
        encrypt: Encrypts the plaintext data of a file and encrypts the file in place.
            Parameters:
            - plaintext (string): Contains the plaintext data from the file that the user wants to encrypt.
            - file_name (string): Contains the data path to the file that the user wants to encrypt.
            
        decrypt: Decrypts the ciphertext data of a file and decrypts the file in place.
            Parameters:
            - ciphertext (string): Contains the ciphertext data from the file that the user wants to decrypt.
            - file_name (string): Contains the data path to the file that the user wants to decrypt.
            
    """
    def __init__(self, key):
        self.key = key
        self.block_size = algorithms.AES.block_size // 8  # Block size in bytes

    def encrypt(self, plaintext, file_name):
        """
            encrypt: Encrypts the plaintext data of a file and encrypts the file in place.
            Parameters:
            - plaintext (string): Contains the plaintext data from the file that the user wants to encrypt.
            - file_name (string): Contains the data path to the file that the user wants to encrypt.
        """
        # Generate a random IV
        iv = os.urandom(self.block_size)
        
        # Define the cipher using the key
        cipher = Cipher(algorithms.AES(self.key), modes.CBC(iv))
        encryptor = cipher.encryptor()

        # Pad the plaintext to ensure it's a multiple of the block size
        padder = padding.PKCS7(algorithms.AES.block_size).padder()
        padded_data = padder.update(plaintext.encode('utf-8')) + padder.finalize()

        # Encrypt the padded data
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()

        # Encode the IV and ciphertext as base64 for safe storage
        iv_b64 = base64.b64encode(iv).decode('utf-8')
        ciphertext_b64 = base64.b64encode(ciphertext).decode('utf-8')

        # Write the encrypted data to the file
        with open(file_name, 'w') as file:
            file.write(ciphertext_b64)
        
        # Write the IV to a separate file
        with open(file_name + '.iv', 'w') as iv_file:
            iv_file.write(iv_b64)
        
        print("File encrypted")

    def decrypt(self, file_name):
        """
            decrypt: Decrypts the ciphertext data of a file and decrypts the file in place.
            Parameters:
            - ciphertext (string): Contains the ciphertext data from the file that the user wants to decrypt.
            - file_name (string): Contains the data path to the file that the user wants to decrypt.
        """
        # Read the IV from the separate file
        with open(file_name + '.iv', 'r') as iv_file:
            iv_b64 = iv_file.read()
            iv = base64.b64decode(iv_b64)

        # Read the ciphertext from the file
        with open(file_name, 'r') as file:
            ciphertext_b64 = file.read()
            ciphertext = base64.b64decode(ciphertext_b64)

        # Define the cipher using the key and the IV
        cipher = Cipher(algorithms.AES(self.key), modes.CBC(iv))
        decryptor = cipher.decryptor()

        # Decrypt the ciphertext
        decrypted_padded_data = decryptor.update(ciphertext) + decryptor.finalize()

        # Unpad the decrypted data to get the original plaintext
        unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
        plaintext = unpadder.update(decrypted_padded_data) + unpadder.finalize()

        # Write the decrypted plaintext back to the file
        with open(file_name, 'w') as file:
            file.write(plaintext.decode('utf-8'))

        print("File decrypted")

In [68]:
def obfuscate_script(file_name):
    """
        obfuscate_script: Obfuscates the file in place.
        Parameters:
        - file_name (string)
    """
    # Open the file and read in the contents
    with open(file_name, 'r') as file:
        script_content = file.read()
        
    # Obfuscate the code using base64
    obfuscated_content = base64.b64encode(script_content.encode('utf-8')).decode('utf-8')
    
    # Reopen the file and write the new contents over the old contents
    with open(file_name, 'w') as file:
        file.write(f'"""{obfuscated_content}"""\n')

    print("File obfuscated")

In [69]:
def deobfuscate_script(file_name):
    # Open the file and read in the contents
    with open(file_name, 'r') as file:
        obfuscated_content = file.read()
        # Extract content between triple quotes
        match = re.search(r'"""(.*?)"""', obfuscated_content, re.DOTALL)
        if match:
            encoded_content = match.group(1)
        else:
            encoded_content = obfuscated_content.strip()

    # Deobfuscate the code using base64 decoder
    script_content = base64.b64decode(encoded_content.encode('utf-8')).decode('utf-8')
    
    # Reopen the file and write the old contents over the obfuscated
    with open(file_name, 'w') as file:
        file.write(script_content)

    print("File deobfuscated")

In [80]:
def main():
    # Enter filename
    file_name = 'test.txt'

    # Prompt the user for the key
    user_key_input = input("Please enter the key: ").strip()
    user_key_bytes = user_key_input.encode('utf-8')

    # Hash the user's input key
    user_key_hash = hashlib.sha256(user_key_bytes).hexdigest()

    # Stored hash of the correct key
    # The hash of 'Big Data Artificial Intelligence' is computed and stored
    stored_key_hash = 'ebc8491ed1b7edda879dc0f54cda8d3075be0f95dd1d69e8340cfa9afcff7341'

    # Compare the hashes
    if user_key_hash != stored_key_hash:
        # If the key was wrong exit the program
        print("Incorrect key. Access denied.")
        exit()

    # Key is correct; proceed with the user's input key
    key = user_key_bytes

    # Choose action based on user input
    print("Please choose from one of the following:")
    print("1: Encrypt and Obfuscate")
    print("2: Encrypt Only")
    print("3: Obfuscate Only")
    print("4: Decrypt and Deobfuscate")
    print("5: Decrypt Only")
    print("6: Deobfuscate Only")
    action = input().strip()

    cipher = AESCipher(key)

    if action == '1':
        # Encrypt and Obfuscate
        # Read the file contents to encrypt
        with open(file_name, 'r') as file:
            file_contents = file.read()

        # Encrypt the file
        cipher.encrypt(file_contents, file_name)

        # Obfuscate the file
        obfuscate_script(file_name)

    elif action == '2':
        # Encrypt Only
        with open(file_name, 'r') as file:
            file_contents = file.read()
        cipher.encrypt(file_contents, file_name)

    elif action == '3':
        # Obfuscate Only
        obfuscate_script(file_name)

    elif action == '4':
        # Decrypt and Deobfuscate
        
        # Deobfuscate the file first
        deobfuscate_script(file_name)
        
        # Then decrypt the file
        cipher.decrypt(file_name)

    elif action == '5':
        # Decrypt Only
        cipher.decrypt(file_name)

    elif action == '6':
        # Deobfuscate Only
        deobfuscate_script(file_name)

    else:
        print("Invalid action. Please enter a number between 1 and 6.")

if __name__ == "__main__":
    main()

Please enter the key: Big Data Artificial Intelligence
Please choose from one of the following:
1: Encrypt and Obfuscate
2: Encrypt Only
3: Obfuscate Only
4: Decrypt and Deobfuscate
5: Decrypt Only
6: Deobfuscate Only


KeyboardInterrupt: Interrupted by user